## Step 1: Load Required Libraries

In [1]:
import json
import chess
import zstandard as zstd
from pathlib import Path
from typing import Dict, List

## Step 2: Load Sample Data

Let's load a few records to understand the structure.

In [2]:
# Load first 100 records for analysis
eval_file = Path("lichess_db_eval.jsonl.zst")

dctx = zstd.ZstdDecompressor()
records = []

with open(eval_file, "rb") as compressed:
    with dctx.stream_reader(compressed) as reader:
        buffer = ""
        chunk_size = 1024 * 1024  # 1MB chunks
        
        while len(records) < 100:
            chunk = reader.read(chunk_size)
            if not chunk:
                break
            
            buffer += chunk.decode('utf-8')
            lines = buffer.split('\n')
            buffer = lines[-1]  # Keep incomplete line
            
            for line in lines[:-1]:
                if line.strip():
                    records.append(json.loads(line.strip()))
                    if len(records) >= 100:
                        break

print(f"Loaded {len(records)} records")
print(f"\nFirst record:")
print(json.dumps(records[0], indent=2)[:500])

Loaded 100 records

First record:
{
  "fen": "7r/1p3k2/p1bPR3/5p2/2B2P1p/8/PP4P1/3K4 b - -",
  "evals": [
    {
      "pvs": [
        {
          "cp": 69,
          "line": "f7g7 e6e2 h8d8 e2d2 b7b5 c4b3 g7f6 d1e1 a6a5 a2a3"
        },
        {
          "cp": 163,
          "line": "h8d8 d1e1 a6a5 a2a3 c6d7 e6e7 f7f6 e1f2 b7b5 c4b3"
        },
        {
          "cp": 229,
          "line": "h8a8 d1e1 a6a5 e6h6 f7g7 h6h4 a8d8 c4d3 c6g2 d3f5"
        },
        {
          "cp": 231,
          "line": "h8f8 d1e1 b7b5 c4b3 a6


## Step 3: Create Rationale Generator

Generate a rationale based on:
- Position evaluation (cp score)
- Best move characteristics
- Comparison with alternatives

In [3]:
def generate_rationale(board: chess.Board, best_pv: Dict, alt_pvs: List[Dict]) -> str:
    """
    Generate a rationale explaining why the best move is best.
    
    This creates training data where the model learns to explain its reasoning.
    """
    # Get the best move
    best_move_uci = best_pv['line'].split()[0]
    best_move = chess.Move.from_uci(best_move_uci)
    best_san = board.san(best_move)
    
    # Get evaluation
    if 'cp' in best_pv:
        best_eval = best_pv['cp']
        eval_str = f"{best_eval/100:+.2f} pawns"
        
        if abs(best_eval) < 50:
            position_type = "equal position"
        elif best_eval > 0:
            position_type = "advantage for White"
        else:
            position_type = "advantage for Black"
    else:
        mate_in = best_pv['mate']
        eval_str = f"mate in {abs(mate_in)}"
        position_type = f"forced checkmate for {'White' if mate_in > 0 else 'Black'}"
    
    # Basic rationale
    rationale_parts = []
    rationale_parts.append(f"The position shows {position_type}.")
    
    # Describe the best move
    piece = board.piece_at(best_move.from_square)
    piece_name = chess.piece_name(piece.piece_type).capitalize()
    
    if board.is_capture(best_move):
        rationale_parts.append(f"The best move is {best_san}, capturing material.")
    elif board.gives_check(best_move):
        rationale_parts.append(f"The best move is {best_san}, giving check.")
    else:
        rationale_parts.append(f"The best move is {best_san}.")
    
    rationale_parts.append(f"This leads to {eval_str}.")
    
    # Compare with alternatives if available
    if len(alt_pvs) > 0 and 'cp' in best_pv and 'cp' in alt_pvs[0]:
        alt_move_uci = alt_pvs[0]['line'].split()[0]
        alt_move = chess.Move.from_uci(alt_move_uci)
        alt_san = board.san(alt_move)
        alt_eval = alt_pvs[0]['cp']
        
        diff = abs(best_eval - alt_eval)
        if diff > 100:  # Significant difference
            rationale_parts.append(f"Alternative moves like {alt_san} are weaker by {diff/100:.2f} pawns.")
    
    return " ".join(rationale_parts)

# Test the rationale generator
test_record = records[0]
test_board = chess.Board(test_record['fen'])
test_best = test_record['evals'][0]['pvs'][0]
test_alts = test_record['evals'][0]['pvs'][1:]

rationale = generate_rationale(test_board, test_best, test_alts)
print("Example rationale:")
print(rationale)

Example rationale:
The position shows advantage for White. The best move is Kg7. This leads to +0.69 pawns.


## Step 4: Format Training Example

Create the complete training format:
- Input: FEN, legal moves, side to move
- Output: Rationale + Best move

In [4]:
def format_training_example(record: Dict) -> Dict:
    """
    Convert raw eval record into training example.
    
    Returns:
        {
            "input": {
                "fen": str,
                "legal_moves": List[str],
                "side_to_move": str
            },
            "output": {
                "rationale": str,
                "move": str
            },
            "metadata": {
                "evaluation": float,
                "depth": int
            }
        }
    """
    # Parse position
    board = chess.Board(record['fen'])
    legal_moves = [move.uci() for move in board.legal_moves]
    side_to_move = "White" if board.turn == chess.WHITE else "Black"
    
    # Get best move and evaluation
    eval_data = record['evals'][0]  # Use first (best) evaluation
    best_pv = eval_data['pvs'][0]
    best_move_uci = best_pv['line'].split()[0]
    
    # Generate rationale
    alt_pvs = eval_data['pvs'][1:]
    rationale = generate_rationale(board, best_pv, alt_pvs)
    
    # Get evaluation score
    if 'cp' in best_pv:
        evaluation = best_pv['cp'] / 100.0  # Convert to pawns
    else:
        # For mate positions, use large score
        evaluation = 100.0 if best_pv['mate'] > 0 else -100.0
    
    return {
        "input": {
            "fen": record['fen'],
            "legal_moves": legal_moves,
            "side_to_move": side_to_move
        },
        "output": {
            "rationale": rationale,
            "move": best_move_uci
        },
        "metadata": {
            "evaluation": evaluation,
            "depth": eval_data['depth'],
            "knodes": eval_data['knodes']
        }
    }

# Test formatting
example = format_training_example(records[0])
print("Training example:")
print(json.dumps(example, indent=2))

Training example:
{
  "input": {
    "fen": "7r/1p3k2/p1bPR3/5p2/2B2P1p/8/PP4P1/3K4 b - -",
    "legal_moves": [
      "h8g8",
      "h8f8",
      "h8e8",
      "h8d8",
      "h8c8",
      "h8b8",
      "h8a8",
      "h8h7",
      "h8h6",
      "h8h5",
      "f7g8",
      "f7f8",
      "f7g7",
      "c6e8",
      "c6d7",
      "c6d5",
      "c6b5",
      "c6e4",
      "c6a4",
      "c6f3",
      "c6g2",
      "b7b6",
      "a6a5",
      "h4h3",
      "b7b5"
    ],
    "side_to_move": "Black"
  },
  "output": {
    "rationale": "The position shows advantage for White. The best move is Kg7. This leads to +0.69 pawns.",
    "move": "f7g7"
  },
  "metadata": {
    "evaluation": 0.69,
    "depth": 46,
    "knodes": 4189972
  }
}


## Step 5: Process Multiple Examples

Format all loaded records into training examples.

In [5]:
# Process all loaded records
training_examples = []

for record in records:
    try:
        example = format_training_example(record)
        training_examples.append(example)
    except Exception as e:
        print(f"Error processing record: {e}")
        continue

print(f"\nCreated {len(training_examples)} training examples")
print(f"Success rate: {len(training_examples)/len(records)*100:.1f}%")


Created 100 training examples
Success rate: 100.0%


## Step 6: Show Example Variations

Display different types of positions and their rationales.

In [6]:
# Show 5 diverse examples
for i, example in enumerate(training_examples[:5], 1):
    print(f"\n{'='*80}")
    print(f"EXAMPLE {i}")
    print(f"{'='*80}")
    
    # Show the position
    board = chess.Board(example['input']['fen'])
    print(f"\nPosition: {example['input']['side_to_move']} to move")
    print(board)
    
    print(f"\nLegal moves: {len(example['input']['legal_moves'])}")
    print(f"Evaluation: {example['metadata']['evaluation']:+.2f} pawns")
    print(f"Depth: {example['metadata']['depth']}")
    
    print(f"\n📝 RATIONALE:")
    print(f"   {example['output']['rationale']}")
    
    print(f"\n♟️  BEST MOVE:")
    best_move = chess.Move.from_uci(example['output']['move'])
    san = board.san(best_move)
    print(f"   {example['output']['move']} ({san})")


EXAMPLE 1

Position: Black to move
. . . . . . . r
. p . . . k . .
p . b P R . . .
. . . . . p . .
. . B . . P . p
. . . . . . . .
P P . . . . P .
. . . K . . . .

Legal moves: 25
Evaluation: +0.69 pawns
Depth: 46

📝 RATIONALE:
   The position shows advantage for White. The best move is Kg7. This leads to +0.69 pawns.

♟️  BEST MOVE:
   f7g7 (Kg7)

EXAMPLE 2

Position: Black to move
. . . . . . . .
. . . . r . . .
. . R . . p k .
. . . . . . p p
. . . P . . . .
. . . . . . P .
. . . . . K . P
. . . . . . . .

Legal moves: 21
Evaluation: +0.00 pawns
Depth: 58

📝 RATIONALE:
   The position shows equal position. The best move is Ra7. This leads to +0.00 pawns.

♟️  BEST MOVE:
   e7a7 (Ra7)

EXAMPLE 3

Position: White to move
. . . . . . k .
. . . . . . p .
. . . . . . . .
. . . . K . . .
. . . . N N . .
. . . . . . . .
. . . . . . . .
. . . . . . . .

Legal moves: 21
Evaluation: +100.00 pawns
Depth: 95

📝 RATIONALE:
   The position shows forced checkmate for White. The best move is Ke6. 

## Step 7: Training Format for Your Model

Show how to format this for actual model training with proper tags.

In [7]:
def format_for_model_training(example: Dict, tokenizer) -> str:
    """
    Format example for your model's training format.
    Uses the same tags as your puzzle training: <rationale> and <uci_move>
    """
    # Create system message
    system_msg = "You are a chess expert. Analyze positions and find the best move with clear reasoning."
    
    # Create user prompt
    user_msg = f"""Analyze this chess position and find the BEST move.

Position (FEN): {example['input']['fen']}
Side to move: {example['input']['side_to_move']}
Legal moves: {' '.join(example['input']['legal_moves'])}

Provide your analysis in <rationale> tags, then the best move in <uci_move> tags."""
    
    # Create assistant response
    assistant_msg = f"""<rationale>{example['output']['rationale']}</rationale>
<uci_move>{example['output']['move']}</uci_move>"""
    
    # Format as conversation
    messages = [
        {"role": "system", "content": system_msg},
        {"role": "user", "content": user_msg},
        {"role": "assistant", "content": assistant_msg}
    ]
    
    # Apply chat template
    formatted = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=False
    )
    
    return formatted

# Show formatted example (without tokenizer, just show structure)
print("\n" + "="*80)
print("FORMATTED FOR TRAINING (without tokenizer)")
print("="*80)

example = training_examples[0]
print(f"\n🔵 INPUT:")
print(f"Position: {example['input']['fen']}")
print(f"Side to move: {example['input']['side_to_move']}")
print(f"Legal moves: {' '.join(example['input']['legal_moves'][:10])}...")

print(f"\n🟢 OUTPUT:")
print(f"<rationale>{example['output']['rationale']}</rationale>")
print(f"<uci_move>{example['output']['move']}</uci_move>")

print(f"\n📊 METADATA:")
print(f"Evaluation: {example['metadata']['evaluation']:+.2f}")
print(f"Depth: {example['metadata']['depth']}")


FORMATTED FOR TRAINING (without tokenizer)

🔵 INPUT:
Position: 7r/1p3k2/p1bPR3/5p2/2B2P1p/8/PP4P1/3K4 b - -
Side to move: Black
Legal moves: h8g8 h8f8 h8e8 h8d8 h8c8 h8b8 h8a8 h8h7 h8h6 h8h5...

🟢 OUTPUT:
<rationale>The position shows advantage for White. The best move is Kg7. This leads to +0.69 pawns.</rationale>
<uci_move>f7g7</uci_move>

📊 METADATA:
Evaluation: +0.69
Depth: 46


## Step 8: Save Training Data

Save the formatted examples to a file for actual training.

In [ ]:
# Save to JSONL
output_file = Path("data/processed/eval_training_examples_100.jsonl")
output_file.parent.mkdir(parents=True, exist_ok=True)

with open(output_file, 'w') as f:
    for example in training_examples:
        f.write(json.dumps(example) + '\n')

print(f"✅ Saved {len(training_examples)} examples to {output_file}")

# Show stats
evals = [ex['metadata']['evaluation'] for ex in training_examples]
print(f"\n📊 Statistics:")
print(f"   Min eval: {min(evals):+.2f}")
print(f"   Max eval: {max(evals):+.2f}")
print(f"   Avg eval: {sum(evals)/len(evals):+.2f}")

depths = [ex['metadata']['depth'] for ex in training_examples]
print(f"   Min depth: {min(depths)}")
print(f"   Max depth: {max(depths)}")
print(f"   Avg depth: {sum(depths)/len(depths):.1f}")

## Summary

This notebook shows how to convert raw Stockfish evaluations into training data:

**Input format:**
- FEN position
- Legal moves
- Side to move

**Output format:**
- Rationale (generated from evaluation)
- Best move (from Stockfish)

**Next steps:**
1. Process full dataset (not just 100 examples)
2. Filter by evaluation quality (depth, knodes)
3. Balance position types (tactical vs positional)
4. Combine with puzzle data for comprehensive training